In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

In [56]:
ratings_df = pd.read_csv('movielens_matrix.csv')
ratings_df.head()

,user id,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
users = ratings_df['user id']
ratings_df = ratings_df.drop('user id',axis=1)
books = ratings_df.columns
print(users[0:10])
print(books[0:10])


0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: user id, dtype: int64
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], dtype='object')


In [59]:
ratings_df.fillna(0, inplace=True)
#ratings_df[ratings_df < 3] = 0
ratings_df = ratings_df.astype(bool)
ratings_df.head()

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Apriori/Inference Rules Algorithm from Scratch

The apriori algorithm works by setting a minimum support threshhold and discarding any transaction sets which fall below this threshold. Checking all transaction sets would be copmutationally expensive. Instead, apriori checks the transaction of just one item at a time. If the single item transaction has sufficient support, then it is saved for later. We then move on to the next single element transaction. If it is sufficient, we save it for later. We then combine the single element transaction with all previously saved transaction subsets, keeping those new subsets which have sufficient support. The process is repeated for each possible item in our transactions. This allows us to find all useful transaction subsets while only passing over the data once.

After finding the transactions with sufficient support, we search for suitable asociation rules, keeping only those rules which surpass a confidence threshold. To create an association rule we simply partition one of the transaction sets found previously into antecedent and consequent sets. We then calculate the confidence for this rule to determine whether we should keep it or not.

#### Support
Support quantifies how often a transaction appears in the data. This helps us to focus on only those transaction for which finding an association rule will be consequential. Support is calculated as the probability of finding the transaction subset being part of one of the given transactions.
$$S(t) = P(t\subseteq T) = \frac{\lvert t \rvert}{\lvert T \rvert}$$

### Confidence
Confidence tells us what you would imagine, how confident we are that it is a good rule. Confidence is calculated as the conditional probability of the consequent happening given that the antecedent occured.
$$C(A \implies B) = P(B\vert A) = \frac{P(B\cap A)}{P(A)}$$

In [60]:
class apriori_rules:
    def __init__(self, support_threshold = 0.8):
        self.support_threshold = support_threshold
        self.confidence_threshold = None
        self.confidence_threshold = None
        self.transaction_subsets = None
        self.supported_t = None
        self.supported_values = None
        self.data = None
        self.itemsets = None
        self.rules = None


    def fit(self, X):
        
        '''Fits the transaction set. Search for transaction subset with sufficient support. 
        Save array/df of transactions and support as class attribute.
        X, our data, is the set of transactions. It should be in the form of rows of transactions.
        For each possible item in a transaction, there should be a column of binary values.'''

        X_array = X.to_numpy()
        self.data = X_array
        X_transpose = X_array.T
        #get number of transactions
        total_transactions = X_array.shape[0]
        #list of transactions which have sufficient support
        self.supported_t = []
        #list of support values for itemsets in supported_t
        self.support_values = []
        #dictionary to hold the chosen itemsets and their associted support values for easy access later
        self.itemsets = {}
        #holds indices of rows which hvae supported item sets
        supported_rows = []
        
        
        #Check each single element set, combine with prvious sets, save sufficent sets
        for index, col in enumerate(X_transpose):
            col_support = col.sum()/total_transactions
            #if not enough support for single element itemset, continue since no other itemset with that item will have enough support
            if col_support < self.support_threshold:
                continue

            #update our list of supported transactions
            copy_supported_t = self.supported_t.copy()
            for t in copy_supported_t:
                mask = X_array[:, tuple(t+[index])].all(axis=1)
                count = mask.sum()
                itemset_support = count/total_transactions
                if itemset_support > self.support_threshold:
                    self.supported_t.append(t + [index])
                    self.support_values.append((itemset_support))
                    self.itemsets[tuple(t + [index])] = itemset_support
             
            #add single element itemset
            self.supported_t.append([index])
            self.support_values.append((col_support))
            self.itemsets[tuple([index])] = col_support

        df_dict = {'itemsets': self.supported_t, 'support':self.support_values}
        self.transaction_subsets = pd.DataFrame(df_dict)

        return

    def find_rules(self, confidence_threshold=0.8):
        self.confidence_threshold = confidence_threshold
        #lists to hold rule information. Will be made into a DataFrame at the end
        antecedent_list = []
        consequent_list = []
        support_list = []
        confidence_list = []
        
        for itemset, support in self.itemsets.items():
            if len(itemset) == 1:
                continue
            #get all combinations of possible combinations of antecedent -> consequent pairs
            #get all possible antecedents of all possible lengths
            for k in range(1,len(itemset)):
                combs = list(combinations(itemset,k))
                for antecedent in combs:
                    #calculate confidence
                    confidence = support/self.itemsets[tuple(antecedent)]
                    if confidence >= confidence_threshold:
                        consequent = tuple(c for c in itemset if c not in antecedent) #find consequent
                        #save to rules dictionary
                        antecedent_list.append(antecedent)
                        consequent_list.append(consequent)
                        support_list.append(support)
                        confidence_list.append(confidence)
            
        #create rules dataframe
        df_dict = {'antecedents': antecedent_list, 'consequents':consequent_list, 'support':support_list,'confidence':confidence_list}
        self.rules = pd.DataFrame(df_dict)
          
        return

In [61]:
apriori_model = apriori_rules(support_threshold=0.2)
apriori_model.fit(ratings_df)
print(apriori_model.transaction_subsets)

apriori_model.find_rules(confidence_threshold = 0.8)
print(apriori_model.rules.head())

        itemsets   support
0            [0]  0.479321
1            [3]  0.221633
2         [0, 6]  0.297985
3            [6]  0.415695
4            [7]  0.232238
...          ...       ...
2886  [257, 747]  0.250265
2887  [287, 747]  0.219512
2888  [293, 747]  0.237540
2889  [299, 747]  0.226935
2890       [747]  0.335101

[2891 rows x 2 columns]
  antecedents consequents   support  confidence
0        (0,)       (49,)  0.404030    0.842920
1      (0, 6)       (49,)  0.258749    0.868327
2        (6,)       (49,)  0.343584    0.826531
3        (7,)       (49,)  0.200424    0.863014
4       (10,)       (49,)  0.222694    0.889831


In [62]:
print(apriori_model.transaction_subsets.shape)
print(apriori_model.rules.shape)

(2891, 2)
(7175, 4)


In [63]:
apriori_model.rules['confidence'].min()

np.float64(0.8)

In [64]:
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [65]:
frequent_itemsets = apriori(ratings_df, min_support=0.2, use_colnames=True)
print(frequent_itemsets.head())
rules = association_rules(frequent_itemsets, metric = 'confidence', min_threshold = 0.8)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())

    support itemsets
0  0.479321      (1)
1  0.221633      (4)
2  0.415695      (7)
3  0.232238      (8)
4  0.317073      (9)
  antecedents consequents   support  confidence      lift
0         (1)        (50)  0.404030    0.842920  1.363420
1       (125)         (1)  0.208908    0.807377  1.684417
2         (4)       (174)  0.200424    0.904306  2.030383
3         (7)        (50)  0.343584    0.826531  1.336910
4         (7)       (100)  0.339343    0.816327  1.515346


In [66]:
print(frequent_itemsets.shape)
print(rules.shape)

(2891, 2)
(7175, 14)


In [67]:
rules['confidence'].min()

np.float64(0.8)

### Make Recommendations based on uesr rating vector

In [68]:
user_ratings = np.zeros(ratings_df.shape[1])
rand_items = np.random.randint(ratings_df.shape[1], size=30)
user_ratings[rand_items] = 1 #random user vector
rated_items = np.argwhere(user_ratings).flatten() #column indeices of books that have been rated
#rated_items = [21,58,64]
print(rand_items)
print(rated_items)

mask = apriori_model.rules['antecedents'].apply(lambda a: set(a).issubset(rated_items)) #indices of antecedents that apply
recs = apriori_model.rules.loc[mask, 'consequents'] #get matching consequents as recommendations
recs = list(set(item for tpl in recs for item in tpl)) #make the recommendations unique
recs = [rec for rec in recs if rec not in rated_items]#remove recommendations matching items already in user list
book_recs = books[recs]#map recs indices back to original book names

print(recs)
print(book_recs)

    
    

[ 556  349 1354 1560  785 1540 1519 1529 1194  106  511 1159 1207 1546
  831  914  440  823  529 1350 1233  368 1655 1110  509  426 1306  937
  428  384]
[ 106  349  368  384  426  428  440  509  511  529  556  785  823  831
  914  937 1110 1159 1194 1207 1233 1306 1350 1354 1519 1529 1540 1546
 1560 1655]
[49, 171, 173]
Index(['50', '172', '174'], dtype='object')
